Trainiert ein Textklassifikationsmodell für sst2. Bei sst2 muss festgestellt werden, ob ein Satz positiv oder negativ ist.

Notebook basiert auf https://github.com/huggingface/setfit/blob/main/README.md

In [1]:
from importlib.resources import files

import polars as pl
from datasets import load_dataset, Dataset
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset

In [ ]:
# Load a dataset from the Hugging Face Hub
dataset = load_dataset("sst2")

# Simulate the few-shot regime by sampling 8 examples per class
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=10, seed=1)
eval_dataset = dataset["validation"].select(range(100))

#Nur zum Anschauen, kann auskommentiert werden
train_dataset.to_polars()

In [ ]:
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    labels=["negative", "positive"],
)

In [ ]:
# Trainingsablauf und Hyperparamter definieren
args = TrainingArguments(
    batch_size=16,
    num_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
    column_mapping={"sentence": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)

In [ ]:
# Train and evaluate
trainer.train()
metrics = trainer.evaluate(eval_dataset)
print(metrics)

In [ ]:
# Run inference
preds = model.predict(["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"])
print(preds)
# ["positive", "negative"]

In [7]:
# Spielt mit den Hyperparametern rum

In [8]:
# Nutzt ein anderes Modell

In [9]:
# Trainiert mit einer GPU auf Google Colab
# TODO

In [ ]:
# Trainiert mit euren eigenen Daten
# Ein Beispiel wie dies aussehen könnte:

df = pl.from_dicts([
    {"sentence": "Beispiel 1, Klasse 1", "label": "Klasse_1"}, #Hier eigene Daten eintragen, ggf. Zeilen hinzufügen
    {"sentence": "Beispiel 2, Klasse 1", "label": "Klasse_1"}, 
    {"sentence": "Beispiel 3, Klasse 2", "label": "Klasse_2"}, 
    {"sentence": "Beispiel 4, Klasse 2", "label": "Klasse_2"}, 
]).sample(shuffle=True, fraction=1, seed=42)
train_dataset = Dataset.from_polars(df)
df

In [11]:
# Trainiert auf einem anderen Datensatz
# Achtung: Column Mapping anpassen